<a href="https://colab.research.google.com/github/mphill82/Coursera_Capstone/blob/main/Clustering_Toronto_Neighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clustering Toronto Neighborhoods by Venues

In [ ]:
import numpy as np
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
import geocoder

### Part 1 - Acquiring Neighborhood information from Wikipedia

First we scrape data from wikipedia on neighborhoods and boroughs of each postal code in Toronto.

In [89]:
page=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(page,'html.parser')

In [90]:
table = soup.find('table')

We extract the postal code, borough, and list of associated neighborhoods from each cell of the table on wikipedia and store it to dataframe df.

In [129]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]   #first three characters in a row
        cell['Borough'] = (row.span.text).split('(')[0]   #everything in the span section of a row before the first
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)   #starting after the first parentheses, remove the rest of the parentheses and replace forward slashes with commas

df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

Finally I sort the dataframe by postalcode and see that there are 103 postal codes.



In [130]:
df=df.sort_values('PostalCode')
df = df.reset_index(drop=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [121]:
df.shape

(103, 3)

### Part 2 - Acquiring Latitude and Longitude data

Geocoder was not working so I used the csv file provided by course instructor to access latitude and longitude data corresponding to toronto postal codes.

In [131]:
#I uploaded the csv file to google drive and made it public
url='https://drive.google.com/file/d/1pwkLe6YsjJjkgamQ5O9eSgzUlp6fAwZs/view?usp=sharing'
file_id=url.split('/')[-2]
#Read the file and convert it to a pandas dataframe
dwn_url='https://drive.google.com/uc?id=' + file_id
df_ll= pd.read_csv(dwn_url)
#Combine this dataframe with the neighborhood dataframe from before
df=pd.concat([df,df_ll[['Latitude','Longitude']]],axis=1)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [132]:
df.shape

(103, 5)

### Part 3 - Clustering with Foursquare Venue data